**Word Sense Disambiguation (WSD)** - это автоматическое определение значения слова через соотнесение слова в контексте с одним из словарных определений этого слова в словаре. Например, у слова "румяна" есть как привычное нам значение "косметика для лица", так и более редкое - "трава, которую использовали для румян". Сейчас мы попробуем автоматически отделить контексты румян как косметики от тех случаев, где, возможно, имеется в виду трава.

In [ ]:
# Данные будем обрабатывать библиотекой Pandas.
import pandas as pd

Для начала используем алгоритм Леска для решения нашей задачи. Согласно алгоритму, если в контексте слова есть слова из одного из его словарных определений, значит, именно это определение и ведет к нужному словарному значению.

In [ ]:
# Возьмем два определения слова "румяна": энциклопедическое определение
# косметического средства с Википедии и определение из лексической
# этноботанической базы данных Фитолекс (Phytolex) https://phytolex.eusp.org/.

# https://ru.wikipedia.org/wiki/%D0%A0%D1%83%D0%BC%D1%8F%D0%BD%D0%B0
definition_rumyana_1 = """
косметическое средство для наведения румянца на щеках. "Косметическая краска для наведения румянца.
Нанести на лицо р. Пользоваться пудрой и румянами". Его наносят в виде пудры или крема.
Используется для придания более свежего и молодого вида и/или чтобы подчеркнуть скулы.
В настоящее время румяна обычно состоят из цветного порошка на основе талька, который наносится кистью на щёки.
В качестве красителя обычно используют сафлор красильный, соединение кармина с гидроксидом аммония, розовую воду, а также другие различные красители.
"""
# https://phytolex.eusp.org/citation/Q2l0YXRpb246MTY2MDc=
definition_rumyana_2 = """
Трава двулѣтная, имѣющая стебель и листы какъ бы щетинами покрытые;
цвѣточки колосомъ разположенные, синенькїе о пяти длинныхъ тычкахъ и одномъ пестикѣ, волосками покрытыхъ.
Изъ подъ кожицы стебля, когда оной переломишь, выступаетъ красной сокъ, коимъ деревенскїя дѣвки румянятся.
Ростетъ въ умѣренной полосе Россїи.
"""

Чтобы получить контексты, в которых мы будем определять, какое из двух значений слова "румяна" используется, идем на сайт Национального корпуса русского языка (НКРЯ) https://ruscorpora.ru/. Контекстов будет много, но, скорее всего, о траве "румяна" раньше говорили чаще - в народной культуре это был основной источник получения косметического средства. Поэтому ограничим наш набор контекстов. Жмякаем на "Основной корпус" и затем рядом со словом "Подкорпус" жмякаем на "Задать". В "Основных параметрах текста" в окне "Дата содания" -> "по" вносим 1900. Это будет самый поздний год создания произведения, из которого мы хотим взять контексты слова "Румяна". Сохраняем подкорпус, нажимая на "Сохранить без просмотра". В новой форме "Слово 1" -> "Лемма" вводим *румяна*.

Слово "румяна" также может быть кратким прилагательным, например в сочетании "девка румяна". В форме "Грамм. признаки" жмякаем "выбрать". Открывается форма, где указаны все возможные грамматические категории слова. Их очень много, а нам нужно удалить именно случаи, где "румяна" - прилагательное. Жмякаем в центре наверху "Инвертировать выбор" - все категории оказались чекнуты. Снимаем галочку с "прилагательное" и "предикатив". Жмякаем "применить". Теперь будут выбраны контексты, в которых румяна, скорее всего, существительное (хотя шум, ошибки тоже могут случаться). Получаем 0 текстов с такими признаками ("По вашему запросу ничего не найдено") - это оттого, что старые тексты в НКРЯ мало размечены. Бесимся, откатываем назад до заводских настроек, т.е. убираем все галочки из "Грамм. признаков". Получаем результат "87 текстов 115 примеров".

В правом верхнем углу поля с контекстами есть магическая кнопочка "Сохранить". Если на нее нажать, то система предложит сначала войти или зарегаться. Я вошла при помощи Яндекс ID. Теперь я могу скачать контексты в форматах Excel, CSV или Word. Вы скачивайте, как вам удобно, а я скачала в .csv. Неочевидно, но... разделитель в этом файле будет ";" - пришлось слегка помучиться, подбирая его, т.к. на сайте я не нашла, где это указано. Мой файлик с контекстами мы сейчас подгрузим.

In [ ]:
# Файл с контекстами я сохранила на своем гугл-диске. Вот так мы его скачиваем в колабе.
!wget 'https://docs.google.com/uc?export=download&id=1ngbMhtZABTxs-DaDk5h91TiHvOQVGDZR' -O 'rumyana_contexts.csv'

--2024-11-12 15:26:57--  https://docs.google.com/uc?export=download&id=1ngbMhtZABTxs-DaDk5h91TiHvOQVGDZR
Resolving docs.google.com (docs.google.com)... 173.194.217.113, 173.194.217.100, 173.194.217.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.217.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ngbMhtZABTxs-DaDk5h91TiHvOQVGDZR&export=download [following]
--2024-11-12 15:26:57--  https://drive.usercontent.google.com/download?id=1ngbMhtZABTxs-DaDk5h91TiHvOQVGDZR&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.141.132, 2607:f8b0:400c:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.141.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136205 (133K) [application/octet-stream]
Saving to: ‘rumyana_contexts.csv’

rumyana_contexts.cs 100%[===================>] 133.01K  --.-

In [ ]:
# Путь к файлу теперь такой.
file_with_contexts = 'rumyana_contexts.csv'

In [ ]:
# Распаковываем цсвшку пандасом.
contexts_pd = pd.read_csv(file_with_contexts, sep=';')

In [ ]:
# Вот и, собственно, контексты.
contexts = contexts_pd[['Left context', 'Center', 'Right context']].fillna('').astype(str).agg(' '.join, axis=1)

Наша задача: отделить контексты, в которых под румянами имеется в виду именно растение (фитоним - sic!), которым человек воспользовался или мог воспользоваться, чтобы украсить лицо, от контекстов, где румяна - это вообще любое косметическое средство. Мы это будем делать при помощи алгоритма Lesk. В великой NLP-библиотеке NLTK этот алгоритм реализован для английского языка. https://www.nltk.org/api/nltk.wsd.lesk.html Алгоритм несложный, так что мы его сейчас реализуем на примере наших румян. Суть его такая: берем полнозначные слова из определений и считаем, сколько из них употреблено в контекстном окне искомого слова. Для первого определения это будут слова "косметический", "препарат", "подкрашивание", "кожа" и т.д. Для второго - "трава", "двулетний", "стебель" и т.д. Однако у нас есть сложность: второе определение и некоторые наши контексты написаны в старой орфографии. Поэтому сначала предобработаем наши тексты.

In [ ]:
# Т.к. наши тексты есть и в старой орфографии, и в современной, приведем их
# к новой орфографии. Для этого заменим некоторые символы. В случае с "ъ"
# кажется, что замена заденет и современные буквы, но это нам не сыграет
# большой роли в дальнейшем.
orfo_change = {'ѣ':'е', 'ї':'и', 'і':'и', 'ъ':''}

In [ ]:
# Сохраним контексты из НКРЯ в отдельную переменную.
c = contexts_pd[['Left context', 'Right context']].fillna('').astype(str).agg(' '.join, axis=1)

In [ ]:
# Используем цикл, чтобы реформировать нашу орфографию - перевести в более
# современный вид.
for oc in orfo_change:
    c = c.str.replace(oc, orfo_change[oc])
    definition_rumyana_1 = definition_rumyana_1.replace(oc, orfo_change[oc])
    definition_rumyana_2 = definition_rumyana_2.replace(oc, orfo_change[oc])

In [ ]:
# Вот так, например, теперь выглядит старинное определение румян.
definition_rumyana_2

'\nТрава двулетная, имеющая стебель и листы как бы щетинами покрытые; \nцветочки колосом разположенные, синенькие о пяти длинных тычках и одном пестике, волосками покрытых. \nИз под кожицы стебля, когда оной переломишь, выступает красной сок, коим деревенския девки румянятся. \nРостет в умеренной полосе России.\n'

In [ ]:
# Для дальнейшей предобработки нам понадобится библиотека NLTK и регулярные выражения.
import nltk

from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language="russian")

nltk.download('stopwords')
from nltk.corpus import stopwords
rus_stop_words = stopwords.words("russian")

import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Эта функция предобработает тексты определений и контекстов -
# приведет их к более однообразной форме.
def preprocess(text):
    # Переведем текст в нижний регистр.
    text = text.lower()
    # Это выражение оставит в тексте только алфавит и цифры.
    text = re.sub(r'[^\w\s-]+', '', text)
    # Теперь удалим стоп-слова из списка токенов.
    clean_text = []
    for token in text.split():
        if token not in rus_stop_words:
            clean_text.append(token)
    # Используем стеммер Snowball библиотеки NLTK, чтобы получить основы
    # слов без окончаний и некоторых суффиксов.
    preprocessed_text = [snowball.stem(t) for t in clean_text]
    return preprocessed_text

In [ ]:
# Вот так будет выглядеть препроцессинг на примере старинной дефиниции слова "румяна".
preprocess(definition_rumyana_2)

['трав',
 'двулетн',
 'имеющ',
 'стебел',
 'лист',
 'щетин',
 'покрыт',
 'цветочк',
 'колос',
 'разположен',
 'синеньк',
 'пят',
 'длин',
 'тычк',
 'одн',
 'пестик',
 'волоск',
 'покрыт',
 'кожиц',
 'стебл',
 'он',
 'перелом',
 'выступа',
 'красн',
 'сок',
 'ко',
 'деревенск',
 'девк',
 'румян',
 'ростет',
 'умерен',
 'полос',
 'росс']

In [ ]:
# Предобработаем наши контексты из НКРЯ.
c_preprocessed = c.apply(preprocess)

In [ ]:
c_preprocessed

,0
0,"[велиш, присматриват, устро, червлениц, белил,..."
1,"[десятк, закрыв, морщин, белил, б, угодн]"
2,"[напрасн, сил, философ, украс, философ, так, б..."
3,"[употреблен, трав]"
4,"[получа, постоянны, сказа, цветны]"
...,...
110,"[крестин, чарк, треб, денежн, вклад, молод, зу..."
111,"[сто, цветны, коробочк, пудр, затейливаг, вид,..."
112,"[перезрел, китаянк, употребля, больш, количест..."
113,"[лиц, подоба, образ, замен, бел, известк, жжен..."


In [ ]:
# Теперь предобработаем два определения и сохраним в новые переменные.
df_1_preprocessed = preprocess(definition_rumyana_1)
df_2_preprocessed = preprocess(definition_rumyana_2)

In [ ]:
# Напишем функцию, которая будет считать число общих слов у контекста и
# у каждого из двух определений. Если их будет больше для первого
# определения, то пусть функция вернет число 1; для второго - 2;
# если их не будет, то 0, а если будет одинаково, то 3.
def get_definition(my_list):
    intersection_1 = [value for value in df_1_preprocessed if value in my_list]
    intersection_2 = [value for value in df_2_preprocessed if value in my_list]
    if intersection_1 > intersection_2:
        return 1
    elif intersection_1 < intersection_2:
        return 2
    elif intersection_1 == 0 and intersection_2 == 0:
        return 0
    elif intersection_1 == intersection_2:
        return 3

In [ ]:
# Обрабатываем наши контексты функцией, получаем числа для каждого контекста.
lesk_rumyana = c_preprocessed.apply(get_definition)
print(lesk_rumyana)

0      1
1      3
2      3
3      2
4      3
      ..
110    1
111    1
112    3
113    1
114    1
Length: 115, dtype: int64


In [ ]:
# Посмотрим, какие контексты относят к первому более привычному определению.
contexts[lesk_rumyana[lesk_rumyana==1].index].to_list()
# Ну, вроде, окей, примеры похожи на то, что речь в них идет именно о косметическом средстве.

['велиш присматриватися, что хорошо устроилася, червленицею ( румянами и белилом лице умастила, чело свое',
 'борозды ея чела, а бѣлила съ  румянами на старомъ лицѣ дѣлаютъ изрядную молодую',
 'отмалевано ее лицо разными притираньями, помадами,  румянами и прочим, и прежде, нежели все',
 'У вас на щеках  румяна на сердце румяна, на совести, румяна',
 'вас на щеках румяна, на сердце  румяна на совести, румяна, на искренности... сажа. ',
 'румяна, на сердце румяна, на совести,  румяна на искренности... сажа. ',
 'отвергая все излишние украшения, или французские  румяна которые человеку с естественным вкусом не',
 'сплетенные из пчелиной шерстки; то, увы!  румяна которые от духу налетали на щечку. ',
 'которой толстые губы и щеки, нащекатуренные  румянами так нравятся многим гуляющим, а более',
 'которые не должны были полинять под  румянами образованности? ',
 'Теперь взошло солнце истины, осветило  румяна на лицах, тление под искусственною жизнью',
 'эгоизме, что очарование мое исчезало

In [ ]:
# Посмотрим на контексты, которые отсылают ко второму определению.
contexts[lesk_rumyana[lesk_rumyana==2].index].to_list()
# Первый пример, конечно, the best. А вот остальные - сомнительно.
# Радует, что здесь примеров меньше, чем для первого определения.

['Употребленіе травы  румянъ ',
 'пѣнѣ съ красной селитры, ни въ  румянахъ которые привозятъ къ намъ изъ Иллиріи',
 'одной свечи заметны были белила и  румяна ',
 'брюнетки ни одной, косы -- крысьи хвосты,  румяна и белила наложены щедро. ',
 'никем не зримый, Досады и стыда  румянами палимый, Искать хотя одной загадочной черты',
 'никем не зримый, Досады и стыда  румянами палимый, Искать хотя одной загадочной черты',
 'Это, отвѣчалъ я, -- въ одной банкѣ  румяна а въ другой бѣлила. ',
 'Съ ея полудѣтскаго лица сошли  румяна оно было грустно и утомлено; казалось',
 'Кроме того, женщины румянятся китайскими  румянами а летом соком земляники. ',
 'сока розовых раковин; кисточкой с этими  румянами Констанций искусно провел по своим смуглым']

Попробуем теперь изменить наш алгоритм вот как. "Румяна" как растение - это фитоним, т.е. слово, которое отсылает к объекту класса "растение". Используем семантические вектора, чтобы определить, какие контексты слова "румяна" по семантике ближе всего к слову "растение".

In [ ]:
# Используем для этого библиотеку spacy https://spacy.io
import spacy
# Команда !python запускает Питон и он скачивает модуль
# для spacy под названием 'ru_core_news_lg'.
# lg - означает large, большая модель.
!python -m spacy download ru_core_news_lg
nlp = spacy.load('ru_core_news_lg')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 49.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Определение слову задавать не будем - обойдемся одной лексемой "растение".
definition_plant = 'растение'

In [ ]:
# Векторизуем сначала наше растение.
nlp_plant = nlp(definition_plant)

In [ ]:
# Напишем функцию, которая будет определять семантическое расстояние от вектора
# лексемы "растение" к вектору каждого контекста - косинусное расстояние. Если расстояние равно 1,
# значит, контекст тождественен слову растение. Чем оно ближе к 1, тем более контекст
# семантически похож на растение, т.е. указывает на фитоним. А чем ближе к 0,
# тем он дальше от фитонима. Отрицательные значения совсем не играют роли.
def cos_sim_plant(text):
    text_nlp = nlp(text)
    return nlp_plant.similarity(text_nlp)

In [ ]:
# Посмотрим на примере нескольких контекстов, какие получаются значения
# косинусного расстояния.
print(cos_sim_plant('пѣнѣ съ красной селитры, ни въ  румянахъ которые привозятъ къ намъ изъ Иллиріи'))
print(cos_sim_plant('Употребленіе травы  румянъ '))
print(cos_sim_plant('как крахмал из сарацинского пшена, а  румяна добываются китайцами из сафлора (carthamus tinctorius). '))
# У первого контекста значение очень близко к 0. Здесь румяна - это просто косметика.
# Второй контекст получил сильно большее значение - по самому контексту видно,
# что здесь речь именно о траве - румяне.
# В третьем контексте тоже упомянуто растение - сафлор. Оно называется не "румяна",
# но оно тоже фитоним. Если бы мы хотели поискать именно фитонимы, то 0.16 -
# значение косинусного расстония для именно третьего контекста, мы бы задали как пороговое.

0.059415570324038144
0.4467041338398088
0.16124935432088472


In [ ]:
# Напишем функцию, которая возвращает число 2, если косинусное расстояние больше
# или равно 0.16, и 1 - если оно меньше.
def cossim_is_plant(text):
    text_nlp = nlp(text)
    sim = nlp_plant.similarity(text_nlp)
    if sim >= 0.16:
        return 2
    else:
        return 1

In [ ]:
# Обработаем наши котнексты этой функцией.
spacy_rumyana = contexts.apply(cossim_is_plant)

<ipython-input-49-1350ee28a211>:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim = nlp_plant.similarity(text_nlp)


In [ ]:
# Посмотрим на потенциальные фитонимы.
contexts[spacy_rumyana[spacy_rumyana==2].index].to_list()
# Контекстов немного. Несколько из них явно отсылают к фитонимам.
# В нескольких есть упоминания рецептов изготовления румян, что тоже неплохо.
# Ну и пара нерелевантных контекстов тоже попала, но это неизбежно.
# Чем выше будет пороговое значение косинусного расстояния, тем больше контекстов отсеется.

['Употребленіе травы  румянъ ',
 'получаютъ постоянныя, и можно сказать, цвѣтныя  румяна ',
 'отвергая все излишние украшения, или французские  румяна которые человеку с естественным вкусом не',
 'Белила и  румяны так же в большом употреблении, но',
 'заставил его отыскать среди банок с  румянами бумажку. ',
 'можно было кокетничать или щеголять, как  румянами и красивыми тряпками, а потому, что',
 'Кроме того, женщины румянятся китайскими  румянами а летом соком земляники. ',
 'сока розовых раковин; кисточкой с этими  румянами Констанций искусно провел по своим смуглым',
 'как крахмал из сарацинского пшена, а  румяна добываются китайцами из сафлора (carthamus tinctorius). ',
 'искусственными приемами красноречия -- как молодая девушка  румянами и белилами. ',
 'перезрелые китаянки употребляют в большом количестве  румяна и белила. ']